In [1]:
import torch
import numpy as np


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

cuda:0
NVIDIA GeForce RTX 3090 Ti


## Модель

https://huggingface.co/Grossmend/rudialogpt3_medium_based_on_gpt2

In [2]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast


checkpoint = "Grossmend/rudialogpt3_medium_based_on_gpt2"   
tokenizer =  GPT2TokenizerFast.from_pretrained(checkpoint)
model = GPT2LMHeadModel.from_pretrained(checkpoint)

model = model.to(device).eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def get_length_param(text: str, tokenizer) -> str:
    tokens_count = len(tokenizer.encode(text))

    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

## Общение с моделью

In [4]:
@torch.inference_mode()
def chat():
    chat_history_ids = torch.zeros((1, 0), dtype=torch.int).to(device)

    human = True

    try:
        while True:
            if human:
                input_user = input("===> Человек: ")
                new_user_input_ids = tokenizer.encode(
                    f"|0|{get_length_param(input_user, tokenizer)}|" + input_user + tokenizer.eos_token,
                    return_tensors="pt"
                ).to(device)
                chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
            else:
                new_user_input_ids = tokenizer.encode(f"|1|{np.random.choice(['2', '3'])}|", return_tensors="pt").to(device)
                chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

                input_len = chat_history_ids.shape[-1]

                chat_history_ids = model.generate(
                    chat_history_ids,
                    num_return_sequences=1,
                    max_length=1024,
                    no_repeat_ngram_size=2,
                    do_sample=True,
                    top_k=50,
                    top_p=0.9,
                    temperature = 0.5,
                    mask_token_id=tokenizer.mask_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    unk_token_id=tokenizer.unk_token_id,
                    pad_token_id=tokenizer.pad_token_id,
                    device=device
                )

                print(f"===> GPT-3:  {tokenizer.decode(chat_history_ids[:, input_len:][0], skip_special_tokens=True)}")

                if chat_history_ids.shape[-1] > 600:
                    return

            human = not human

    except KeyboardInterrupt:
        return

In [5]:
chat()

===> Человек: Расскажи мне анекдот про жабу, пожалуйста.
===> GPT-3:  Жаба на берегу реки, а рядом мужик с удочкой. Жаба: -Мужик, дай удочку, я тебе червячка поймаю. Мужик: Жабы, это не для вас, мы рыбаки. И тут из-за кустов выходит мужик и говорит: "Я тебе на червячке поймаюсь".
===> Человек: Смешной, спасибо!
===> GPT-3:  Да не за что. Мне тоже нравится. А еще я знаю, что это анекдот. Но не могу вспомнить, где я его слышал. :)


In [6]:
chat()

===> Человек: Расскажи анекдот про часы и мужика.
===> GPT-3:  Мужик в магазине часов, а продавец в это время его спрашивает: - А что вы хотите за часы? - Часы, часы, Карл!
===> Человек: Не смешной, давай другой анекдот
===> GPT-3:  В магазине одежды мужик спрашивает у продавца: "А что у вас есть?" - часы. - Ну часы есть, но есть и часы! - Но часы-то есть! А есть ли у Вас часы?"


In [7]:
chat()

===> Человек: Расскажи о себе, откуда ты?
===> GPT-3:  Вроде бы из Беларуси, но не уверен. Может из России. Но это не точно.
===> Человек: А я из Москвы.
===> GPT-3:  Ну, я тоже из Мск, а в чем суть твоего поста? Я вот тоже хочу в Москву, что бы в метро не стоять и на работу не ездить. Вот у меня есть работа, дом, машина, квартира, и я не хочу никуда переезжать. Я не понимаю в чём проблема? Почему ты не можешь просто переехать в другую страну? Или может быть ты из Казахстана? Может быть из Узбекистана? Не знаю, просто хочу переехать. А если ты про Москву и Россию, то это ты зря, тут не все так однозначно. У меня вот, например, есть знакомый, который живет в Москве, у него есть квартира в МСК и машина в Питере, он работает в России, живет один и все у них хорошо. Просто он не хочет никуда уезжать, вот и всё. Что я делаю не так? И почему я должен переплачивать за это? А может я просто не могу? Это же не моя проблема. Ну и конечно же, мне не нравится то, как живут в


In [8]:
chat()

===> Человек: Как ты думаешь, зачем мы на планете Земля?
===> GPT-3:  Ну, я думаю, что это не совсем то, о чём я думал. Как бы, мы - это всё человечество. А все люди - братья. И мы все - люди. Но мы не все, а лишь малая часть. Мы не люди, но мы люди и мы братья, и нам не надо быть братьями, чтобы быть людьми.
===> Человек: Очень глубоко, я ничего не понял.
===> GPT-3:  А что тут понимать? Всё просто: мы часть всего человечества. Это не так просто, как кажется. Люди - не часть человечества, это просто люди


In [9]:
chat()

===> Человек: Кто такой Александр Сергеевич Пушкин?
===> GPT-3:  Это тот, кто написал "Сказку о попе и работнике его Балде"
===> Человек: А что ты еще о нем знаешь?
===> GPT-3:  Ну, он же написал сказку "О попе, работящемся на галерах". А еще он был членом кружка "Арифметика" и читал курс "Буржуазная этика". И еще, у него были часы, которые показывали время по солнцу.
===> Человек: А когда он умер?
===> GPT-3:  Не знаю, как сейчас, но в 1835 году. Так как он написал много стихов, то его смерть была не очень приятной. Но, если говорить о его жизни, его жизнь была очень интересной, и он очень много путешествовал. Я думаю, что он жил бы очень долго, не будь его так много в России. Он был очень популярен. И, думаю что его любили бы. В общем, я думаю он прожил бы не мало лет. А, может быть, даже и больше. Не знаю. По крайней мере, мне так кажется. Это все, о чем я могу сказать. Хотя, это не точно. )


In [10]:
chat()

===> Человек: Как ты думаешь, пойдет ли завтра дождь?
===> GPT-3:  Я думаю, что нет. Дождь идет, но не так часто, как хотелось бы. В основном идет дождь из-за того,что в городе очень много людей,которые не могут выйти на улицу. Но в целом, да, погода сегодня хорошая.
===> Человек: Какое сегодня число?
===> GPT-3:  Сегодня в Москве было +3, а в Питере +2. Сейчас дождь, и, если верить прогнозу, будет идти до конца недели. Надеюсь, завтра будет получше. :-)


In [12]:
chat()

===> Человек: Расскажи мне анекдот про Штирлица, пожалуйста.
===> GPT-3:  Штирлиц приходит к Мюллеру и спрашивает: - Мюллера, а что такое "Штрилиц"? - Штрилица - это когда ты приходишь в бар и заказываешь два пива, но не заказывает два стула. - А что это такое - "штрилице"?
